In [1]:
def count_unique(string):
    counted = dict()
    for char in set(string):
        counted[char] = string.count(char)
    return counted

In [266]:
def read_csv(file):
    import csv
    
    with open(file, 'r', encoding='UTF-8') as file:
        data = csv.reader(file)
        headers = []
        content = []
        for i, line in enumerate(data):
            if i > 0:
                content.append(line)
            else:
                headers = line
        
        return headers, content

In [267]:
result = read_csv('titanic.csv')
print(len(result[0]) == len(result[1][0]))

True


In [270]:
result = read_csv('Киновзгляды.csv')
print(len(result[0]) == len(result[1][0]))

True


### интересные зависимости: влияние пунктов опроса на препочитаемые жанры фильмов

In [19]:
import numpy as np
import pandas as pd
from sklearn.tree import DecisionTreeClassifier

In [20]:
data = pd.read_csv('Киновзгляды.csv', index_col='Отметка времени')

In [66]:
quiz = ["Мальчик или девочка?", "Как часто тебя били в детстве?",
        "Оцените ваш уровень вегетарианца по 5-ти бальной шкале", "Что такое два.ч?",
        "Твоя ориентация?", "Погулять вечером после тяжкого рабочего дня, или посычевать дома?", 
        "Как часто играешь в игры?", "Сериалы?", "А книги? ", "Вы таки антисемит? ",
        "Какой предмет ближе всего по душе?", "Как вы относитесь к аннексии Крыма?", 
        "Владимир Путин - ", "Как хорошо вы знаете английский?", "Было ли раньше лучше? ", 
        "А какие жанры фильмов любишь? "]
genres = ['Боевик', 'Детектив', 'Аниме', 'Драма', 'Комедия', 'Мелодрама', 'Исторический фильм', 'Трагедия', 
          'Фильм ужасов', 'Фантастика']

In [55]:
def convert_to_dict(x):
    dictionary = {}
    for item in x:
        dictionary[item] = 1
    return dictionary

In [226]:
x = data.get(quiz_head)

x.update(x[quiz[0]].apply(lambda x: 0 if x == 'Девочка' else 1))
x.update(x[quiz[1]].apply(lambda x: 0 if x == 'Не били' else 1 if x == 'Разве что по делу' else 2))
x.update(x[quiz[3]].apply(lambda x: 
                          0 if x == 'Не знаю' else 
                          1 if x == 'Паблик в вк' else 
                          2 if x == 'Лучшая борда рунета' else 3))
x.update(x[quiz[4]].apply(lambda x: 0 if x == 'Би' else 1 if x == 'Гетеро' else -1))
x.update(x[quiz[5]].apply(lambda x: 1 if x == 'Гулять' else 0))
x.update(x[quiz[9]].apply(lambda x: 0 if x == 'Нет' else 2 if x == 'Да' else 1))
x.update(x[quiz[11]].apply(lambda x: 1 if x == 'Положительно' else -1 if x == 'Отрицательно' else 0))
x.update(x[quiz[13]].apply(lambda x: 
                           0 if x == 'Оригатоё гайзаймас' else 
                           1 if x == 'Чуть лучше, чем ничего' else 
                           2 if x == 'Нормально' else
                           3 if x == 'Хорошо' else
                           4 if x == 'Отлично' else 5))
x.update(x[quiz[14]].apply(lambda x: 0 if x == 'Нет' else 1))

x = pd.get_dummies(x, columns=[quiz[10], quiz[12]])

liked_genres = x.get([quiz[0], quiz[-1]])[quiz[-1]].str.split(';')
liked_genres = liked_genres.apply(lambda x: convert_to_dict(x)).apply(pd.Series).fillna(0)

x = x.drop(quiz[-1], axis=1)

In [246]:
def count_impact(genre, liked_genres):
    clf = DecisionTreeClassifier()
    clf.fit(x, liked_genres[genre])
    res = clf.feature_importances_

    arr_of_impacts = {}
    for quest, out in zip(x.columns, res):
        arr_of_impacts[quest] = f'{out * 100}'

    return arr_of_impacts

In [224]:
def print_array(arr):
    for quest, out in sorted(arr.items(), key=lambda x: float(x[1]), reverse=True):
        if out > 2: 
            out = f'{out:.2f}'
            connect = ' ' * (85 - len(quest) - len(out))
            print(quest, out + '%', sep=connect)
    print()

### запускать это ↓

In [277]:
for genre in genres:
    result = {item: 0.0 for item in x.columns}
    
    iterations = 20
    for i in range(iterations):
        array = count_impact(genre, liked_genres)
        for quest, out in array.items():
            result[quest] += float(out)
    
    for key, val in result.items():
        result[key] /= iterations
        
    param = 'Параметр'    
    impact = 'Влияние на выбор жанра'
    
    print(genre.rjust(86 // 2, ' '))
    print(param + impact.rjust(85 - len(param) + 1, ' '), 
          len(param) * '-' + (len(impact) * '-').rjust(85 - len(param) + 1, ' '), sep='\n')
    print_array(result)

                                     Боевик
Параметр                                                        Влияние на выбор жанра
--------                                                        ----------------------
Сериалы?                                                                        16.75%
Как часто играешь в игры?                                                       11.95%
А книги?                                                                        10.51%
Какой предмет ближе всего по душе?_Информатика                                   7.05%
Оцените ваш уровень вегетарианца по 5-ти бальной шкале                           5.86%
Какой предмет ближе всего по душе?_История                                       4.11%
Какой предмет ближе всего по душе?_Иностранные языки                             4.04%
Погулять вечером после тяжкого рабочего дня, или посычевать дома?                3.89%
Владимир Путин - _-                                                              3.64%

                         Исторический фильм
Параметр                                                        Влияние на выбор жанра
--------                                                        ----------------------
Сериалы?                                                                        19.75%
Как хорошо вы знаете английский?                                                10.76%
А книги?                                                                         9.99%
Как вы относитесь к аннексии Крыма?                                              5.83%
Какой предмет ближе всего по душе?_История                                       5.52%
Какой предмет ближе всего по душе?_Матеша                                        4.80%
Оцените ваш уровень вегетарианца по 5-ти бальной шкале                           4.62%
Как часто играешь в игры?                                                        3.59%
Твоя ориентация?                                                                 3.43%